#### Test att hämta bilder och söka i SPA
* [Denna notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20Litteraturbanken.ipynb) 

Tanken att se vilka möjligheter som finns
* skapa ett script --> ett klick för att ladda upp bilder se test [spa2commons](https://commons.wikimedia.org/wiki/User:Salgo60/spa2commons.js) - GITHUB [salgo60/spa2Commons](https://github.com/salgo60/spa2Commons) 
* att ta en plats som en kyrkogård eller en koppling Litteraturbanken och se vad som kan matchas
* att stämma av länkar SPA -> WIkipedia att dessa finns i WIkidata med länk tillbaka via [P4819](https://www.wikidata.org/wiki/Property:P4819?uselang=sv)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-11-16 19:35:11.686486


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/latest.php"
url= "https://xn--portrttarkiv-kcb.se/endpoints/search.php"
url= "https://portrattarkiv.se/endpoints/search.php"  


## Icke kopplade personer SPA P4819 hos Litteraturbanken P5101

Testar att söka med mer parametrar för att få bättre precision i rankingen
* personer hos Litteraturbanken [Property:P5101](https://www.wikidata.org/wiki/Property:P5101?uselang=sv)


In [9]:
# query to get people with no picture in Litteraturbanken
endpoint_url = "https://query.wikidata.org/sparql"
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

query = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDayWD))) AS ?search)
?birthDayWD
?firstnameLabel ?lastnameLabel
?item ?itemLabel 
(str(year(?birthDayWD)) AS ?BirthYearWD)
  WHERE {
  ?item wdt:P5101 ?Litt. 
  minus {?item wdt:P4819 ?c} 
  minus {?item wdt:P18 ?d }
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} """

# query for all Littbank without a picture in WD but article i Wikidata
queryArticle = """SELECT distinct (CONCAT (?itemLabel," ",str(year(?birthDayWD))) AS ?search)
?birthDayWD
?firstnameLabel ?lastnameLabel
?item ?itemLabel ?wikipedia
(str(year(?birthDayWD)) AS ?BirthYearWD)
  WHERE {
  ?item wdt:P5101 ?Litt. 
  minus {?item wdt:P4819 ?c} 
  minus {?item wdt:P18 ?d }
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  {
      ?wikipedia schema:about ?item .
      ?wikipedia schema:inLanguage "sv" .
      ?wikipedia schema:isPartOf <https://sv.wikipedia.org/> .
    }  

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


#SPALittdf = get_sparql_dataframe(endpoint_url, query)
SPALittdf = get_sparql_dataframe(endpoint_url, queryArticle)
SPALittdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   search          504 non-null    object
 1   birthDayWD      505 non-null    object
 2   firstnameLabel  495 non-null    object
 3   lastnameLabel   341 non-null    object
 4   item            518 non-null    object
 5   itemLabel       518 non-null    object
 6   wikipedia       518 non-null    object
 7   BirthYearWD     504 non-null    object
dtypes: object(8)
memory usage: 32.5+ KB


In [10]:
SPALittdf

,search,birthDayWD,firstnameLabel,lastnameLabel,item,itemLabel,wikipedia,BirthYearWD
0,Abraham Magni Sahlstedt 1716,1716-05-20T00:00:00Z,Abraham,None,http://www.wikidata.org/entity/Q6088609,Abraham Magni Sahlstedt,https://sv.wikipedia.org/wiki/Abraham_Magni_Sa...,1716
1,Adam av Bremen 1100,1100-01-01T00:00:00Z,Adam,None,http://www.wikidata.org/entity/Q109244,Adam av Bremen,https://sv.wikipedia.org/wiki/Adam_av_Bremen,1100
2,Adil Bergström 1865,1865-01-01T00:00:00Z,Adil,Bergström,http://www.wikidata.org/entity/Q16945104,Adil Bergström,https://sv.wikipedia.org/wiki/Adil_Bergstr%C3%B6m,1865
3,Adolf Fredrik Ristell 1744,1744-07-17T00:00:00Z,Adolf,None,http://www.wikidata.org/entity/Q6073623,Adolf Fredrik Ristell,https://sv.wikipedia.org/wiki/Adolf_Fredrik_Ri...,1744
4,Aegidius Girs 1583,1583-01-01T00:00:00Z,Aegidius,Girs,http://www.wikidata.org/entity/Q5760312,Aegidius Girs,https://sv.wikipedia.org/wiki/Aegidius_Girs,1583
...,...,...,...,...,...,...,...,...
513,Wilhelm Julius Coyet 1647,1647-04-14T00:00:00Z,Wilhelm,None,http://www.wikidata.org/entity/Q2573497,Wilhelm Julius Coyet,https://sv.wikipedia.org/wiki/Wilhelm_Julius_C...,1647
514,Wilhelm Kruse 1674,1674-01-01T00:00:00Z,Wilhelm,Kruse,http://www.wikidata.org/entity/Q5924689,Wilhelm Kruse,https://sv.wikipedia.org/wiki/Wilhelm_Kruse,1674
515,Ylva Hellerud 1955,1955-04-14T00:00:00Z,Ylva,Hellerud,http://www.wikidata.org/entity/Q29885782,Ylva Hellerud,https://sv.wikipedia.org/wiki/Ylva_Hellerud,1955
516,Zacharias Cajander 1808,1808-04-08T00:00:00Z,Zacharias,Cajander,http://www.wikidata.org/entity/Q15731822,Zacharias Cajander,https://sv.wikipedia.org/wiki/Zakarias_Cajander,1808


In [11]:
from wikidata.client import Client
from tqdm.notebook import tqdm 
import urllib3, json
http = urllib3.PoolManager() 
SPAdetail = "https://portrattarkiv.se/details/"
url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

def getdfScore(panddf):
    dfScore = pd.DataFrame(columns=['all','wd','spaid','score',
                                    'FirstNameWD','LastNameWD','BirthDayWD',
                                    'FirstNameSPA','LastNameSPA','BirthDaySPA'])
    #for index, row  in SPALittdf.iterrows():
    for index, row in tqdm(panddf.iterrows(),total=panddf.shape[0]):
    
        allString=row["search"]
        wd=row["item"]
        print("\n",allString,wd)
        encoded_body = json.dumps({
            "limit": "5",
            "from": "0",
            "firstname" : row["firstnameLabel"], 
            "lastname" : row["lastnameLabel"], 
            "birthyear":row["BirthYearWD"],            
            "all":allString
            
        })
    
        r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

        if r.status != 200:
            print(r.status)
            continue
    
        data = json.loads(r.data.decode('utf-8'),) 
        urls = []

        for h in data["hits"]["hits"]:
            spaid = h["_id"]
            #print(h)
            source = h["_source"]
            try:
                urlPicture = urlbasePic + spaid
                urls.append(urlPicture)
                score = h["_score"]
                FirstNameSPA = source["FirstName"]
                LastNameSPA = source["LastName"]
                BirthYearSPA = source["BirthYear"]
                print("\t\t",score,FirstNameSPA, " ", LastNameSPA, " - ", BirthYearSPA,SPAdetail+spaid, "\t", )
                dfScore = dfScore.append({
                'all' : allString,
                'wd' : wd,
                'spaid' : spaid,
                'score' : score,
                'FirstNameWD' : row["firstnameLabel"],
                'LastNameWD' : row["lastnameLabel"],
                'BirthYearWD' : row["BirthYearWD"],
                'FirstNameSPA' : FirstNameSPA,
                'LastNameSPA' : LastNameSPA,
                'BirthYearSPA' : BirthYearSPA
                },ignore_index=True)
            except Exception as e:
                print(f"{type(e).__name__} at line {e.__traceback__.tb_lineno} of {__file__}: {e}")
    return dfScore

In [ ]:
dfScoreSPALitt = getdfScore(SPALittdf)
dfScoreSPALitt.info()

  0%|          | 0/518 [00:00<?, ?it/s]


 Abraham Magni Sahlstedt 1716 http://www.wikidata.org/entity/Q6088609

 Adam av Bremen 1100 http://www.wikidata.org/entity/Q109244

 Adil Bergström 1865 http://www.wikidata.org/entity/Q16945104
		 15.148095 Axel   Engström  -  1865 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABkJpA 	
		 14.953324 Axel   Engström  -  1865 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAABApQ 	

 Adolf Fredrik Ristell 1744 http://www.wikidata.org/entity/Q6073623

 Aegidius Girs 1583 http://www.wikidata.org/entity/Q5760312

 Alexander Hacks 1700 http://www.wikidata.org/entity/Q5781389

 Alf Önnerfors 1925 http://www.wikidata.org/entity/Q1237808
		 30.29434 Alf   Skagerström  -  1925 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAACmXw 	
		 28.389706 Alf   Nordin  -  1925 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABWPPg 	
		 28.318495 Alf   Nordin  -  1925 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAABwmQ 	
		 27.923626 Alf   Olsson  -  1925 https://portrattarkiv.se/details/sj9PGLAl


 Björn Hedlund 1932 http://www.wikidata.org/entity/Q27131226

 Björn Lundegård 1927 http://www.wikidata.org/entity/Q5966289

 Björn Lundegård 1927 http://www.wikidata.org/entity/Q5966289

 Björn Lundegård 1927 http://www.wikidata.org/entity/Q5966289

 Björn Meidal 1948 http://www.wikidata.org/entity/Q18638928

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Bo Bennich-Björkman 1926 http://www.wikidata.org/entity/Q5570680
		 28.9953 Bo   Långström  -  1926 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAACq_g 	
		 27.207138 Bo   Andersson  -  1926 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA4-DQ 	
		 26.758883 Bo   Kjäll  -  1926 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVYdQ 	
		 26.600145 Bo   Åkerrén  -  1926 https://portrattarkiv.se/details/xnPIH5lXePAAAAAAAAAKWg 	
		 26.511984 Bo   Aler  -  1926 https://portrattarkiv.se/details/sj9PGLAlnm

		 24.293665 Carl   Sundberg  -  1817 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABlJA 	

 Carl Wilhelm Törnegren 1817 http://www.wikidata.org/entity/Q6218100
		 29.861494 Carl   Hjort  -  1817 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAnOg 	
		 28.388618 Carl   Hjort  -  1817 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAPEQ 	
		 27.796207 Erik   af Edholm  -  1817 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAABn6w 	
		 27.334114 Wilhelm   Ågren  -  1817 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABGBpw 	
		 27.19128 Wilhellm   Karsten  -  1817 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUlKQ 	

 Carl Zetterström 1950 http://www.wikidata.org/entity/Q6252273

 Carolina Almqvist 1829 http://www.wikidata.org/entity/Q4935231

 Casper Christoffer Hanell 1746 http://www.wikidata.org/entity/Q49104451

 Charlotta Frölich 1698 http://www.wikidata.org/entity/Q13565189

 Charlotta Öberg 1818 http://www.wikidata.org/entity/Q4991091

 Christer Åsberg 1940 ht

In [7]:
dfScoreSPALitt.sort_values('score', ascending=False)
dfScoreSPALitt.groupby("wd")

In [8]:
#Group by WIkidata Q number and get highest score 

#create a helper column with max
dfScoreSPALitt['max'] = dfScoreSPALitt.groupby('wd')['score'].transform('max')  

dfScoreSPALittMax =  dfScoreSPALitt.groupby('wd')['score','max','spaid','FirstName','LastName','BirthYear'].apply(lambda x: x.nlargest(1, columns=['score'])).sort_values(['max','wd','score'], ascending=False)
dfScoreSPALittMax   


<ipython-input-8-53d508d08f0b>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfScoreSPALittMax =  dfScoreSPALitt.groupby('wd')['score','max','spaid','FirstName','LastName','BirthYear'].apply(lambda x: x.nlargest(1, columns=['score'])).sort_values(['max','wd','score'], ascending=False)


KeyError: "Columns not found: 'LastName', 'BirthYear', 'FirstName'"

In [ ]:
dfScoreSPALittMax.to_csv("Littraturbanken_max.csv") 